### SNN accelerator BP part

In [1]:
import torch
import torch.nn as nn
import numpy as np

In [2]:
device = torch.device("cpu")
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
### initialize the activation function
def activ_func(U):
    return torch.ge(U,0).to(torch.float)

### initialize the dereivative function


def PoissonGen(inp, rescale_fac=2.0):
    rand_inp = torch.rand_like(inp)
    return torch.mul(torch.le(rand_inp * rescale_fac, torch.abs(inp)).float(), torch.sign(inp))

def loss(inp):
    
    num_examples = inp.shape[0]
    # get unnormalized probabilities
    exp_scores = np.exp(inp)
    # normalize them for each example
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

In [103]:
x = torch.rand(1,4)
inp = torch.rand(8,7)
x = x.expand(inp.size()[-1],-1)
x.size()

torch.Size([7, 4])

In [ ]:
class Fully_Connected_Layer(nn.Module):
    def __init__(self, size):
        super(Fully_Connected_Layer, self).__init__()
        
        self.weight_list = [torch.rand(1,size)]
        self.regs_U = [torch.zeros()]
        self.regs_U_grad = []
        self.regs_S = []
    
    def forward(self,inp):
        

In [97]:
class SNN_MNIST_sim(nn.Module):
    def __init__(self):
        super(SNN_MNIST_sim, self).__init__()

        self.time_step = 5 # 5 time steps
        
        self.weight_list=[torch.rand([3,4],dtype=torch.float,device = device)*2-1,
                        torch.rand([4,4],dtype=torch.float,device = device)*2-1,
                        torch.rand([4,2],dtype=torch.float,device = device)*2-1]
        self.regs_U = []
        self.regs_U_grad = []
        self.regs_S = []
        
        for i in range(self.time_step):
            
            self.regs_U.append([torch.zeros([4],dtype=torch.double,device=device),
                          torch.zeros([4],dtype=torch.double,device=device),
                          torch.zeros([2],dtype=torch.double,device=device)])

            self.regs_U_grad.append([torch.zeros([4],dtype=torch.double,device=device),
                          torch.zeros([4],dtype=torch.double,device=device),
                          torch.zeros([2],dtype=torch.double,device=device)])

            self.regs_S.append([torch.zeros([3],dtype=torch.double,device=device),
                          torch.zeros([4],dtype=torch.double,device=device),
                          torch.zeros([4],dtype=torch.double,device=device)])


        ### initialize other parameters, the activation function, and its derivative
        self.th_f = 0
        self.a_leak = 1
        self.k = 1
        self.target = torch.tensor([1])
        self.criterion = nn.CrossEntropyLoss()
        self.loss = 0
    
    def forward(self, inp):
               
        U_prev_time = self.regs_U[0]
        S_prev_time = self.regs_S[0]
        
        for t in range(self.time_step):
            
            spike_inp = PoissonGen(inp)
            
            # Input layer
            self.regs_S[t][0].data = spike_inp
            spatial_prev = torch.matmul(self.regs_S[t][0],self.weight_list[0])
            
            # Hidden layers
            for l in range(len(self.weight_list)-1):
                
                self.regs_U[t][l].data = self.a_leak*U_prev_time[l]*(1-S_prev_time[l+1]) + spatial_prev
                self.regs_S[t][l+1].data = activ_func(self.regs_U[t][l]-self.th_f)
                spatial_prev = torch.matmul(self.regs_S[t][l+1],self.weight_list[l+1])
                
            # Output layer
            self.regs_U[t][-1].data = U_prev_time[-1] + spatial_prev
            self.regs_U_grad[t][-1].data = abs(self.regs_U[t][-1]-self.target)
            m = nn.Softmax(dim=1)
            output = m(self.regs_U[t][-1])
            
            self.loss = self.criterion(output, self.target)
            U_prev_time = self.regs_U[t]
            S_prev_time = self.regs_S[t]
                    
        return output
    
    
    def backward(self):
        
        U_grad_later_time = self.regs_U_grad[0] # init to zeros
        loss = torch.tensor([0])
        
        for t in range(self.time_step-1,-1,-1):
            
            # Output layer
            self.regs_U_grad[t][-1].data = loss
            
            # Hidden layers
            for l in range(len(self.weight_list)-1,-1,-1):
                print(l)
                
                
                
                self.weight_list[l] = self.weight_list[l] + torch.matmul(torch.transpose(self.regs_U_grad[t][l],0,1),self.regs_S[t][l])
                
            
            

### Forwarding test

In [98]:
inp = torch.rand(1,3)
tester = SNN_MNIST_sim()

out = tester(inp)
tester.backward()

2


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [74]:
a = torch.tensor([[1,2,3,4]])
b = torch.transpose(torch.tensor([[1,0]]),0,1)
torch.matmul()


RuntimeError: size mismatch, m1: [1 x 4], m2: [2 x 1] at C:\w\1\s\tmp_conda_3.7_100118\conda\conda-bld\pytorch_1579082551706\work\aten\src\TH/generic/THTensorMath.cpp:136

In [72]:
b = torch.transpose(torch.tensor([[1,0]]),0,1)
b

tensor([[1],
        [0]])

In [273]:
def activ_func(U):
    return torch.gt(U,0)

def de_func(U):
    k=1
    U = 1 - k*abs(U)
    U[U<0]=0
    return U

In [302]:
inp = torch.rand(1,28,28)
inp = inp.view(1,-1)
time_step = 5
leak = 0.8

s_regs_inp = torch.zeros(time_step,inp.size()[0],inp.size()[-1])

fc_1 = nn.Linear(28*28,4)
u_regs_1 = torch.zeros(time_step,inp.size()[0],fc_1.out_features)
du_regs_1 = torch.zeros(time_step,inp.size()[0],fc_1.out_features)
s_regs_1 = torch.zeros(time_step,inp.size()[0],fc_1.out_features)

fc_2 = nn.Linear(4,6)
u_regs_2 = torch.zeros(time_step,inp.size()[0],fc_2.out_features)
du_regs_2 = torch.zeros(time_step,inp.size()[0],fc_2.out_features)
s_regs_2 = torch.zeros(time_step,inp.size()[0],fc_2.out_features)

fc_out = nn.Linear(6,2)
u_regs_out = torch.zeros(time_step,inp.size()[0],fc_out.out_features)
du_regs_out = torch.zeros(time_step,inp.size()[0],fc_out.out_features)

accum = torch.zeros(inp.size()[0],fc_out.out_features)

for t in range(time_step):
    spike_inp = PoissonGen(inp)
    s_regs_inp.data[t] = spike_inp
    
    U_1 = fc_1(spike_inp)
    u_regs_1.data[t] = leak*u_regs_1[t-1]*(1-s_regs_1[t-1]) + U_1
    s_regs_1.data[t] = activ_func(u_regs_1[t]-0.3)
    
    U_2 = fc_2(U_1)
    u_regs_2.data[t] = leak*u_regs_2[t-1]*(1-s_regs_2[t-1]) + U_2
    s_regs_2.data[t] = activ_func(u_regs_2[t]-0.3)
    
    out = fc_out(U_2)
    u_regs_out.data[t] = u_regs_out[t-1] + out

m = nn.Softmax(dim=1)
output = m(u_regs_out[-1]/time_step)
output

tensor([[0.3984, 0.6016]])

In [317]:
target = torch.tensor([0])
criterion = nn.CrossEntropyLoss()
loss = criterion(output,target)
loss

tensor(0.7999)

In [321]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        
        self.fc_1 = nn.Linear(28*28,4)
        self.fc_2 = nn.Linear(4,6)
        self.fc_out = nn.Linear(6,2)
        self.lif1 = LIF()
        self.lif2 = LIF()
        
    def forward(self, x, t):
        x = self.fc_1(x)
        x = self.lif1(x, t)
        x = self.fc_2(x)
        x = self.lif2(x, t)
        x = self.fc_out(x)
        
        return x

In [326]:
class LIF(nn.Module):
    def __init__(self):
        super(LIF, self).__init__()
        
        self.u_regs = None
        self.du_regs = None
        self.s_regs = None
        self.leak = 0.8
        self.time_steps = 5
        self.thresh = 0.5
        
    def forward(self,inp, t):
        if t == 0:
            size = inp.shape
            self.u_regs = torch.zeros_like(inp).expand(self.time_steps, *size)
            self.u_regs[0] = inp
            self.s_regs = torch.zeros_like(inp).expand(self.time_steps, *size)
            vol = inp - self.thresh
            spike = spike_function(vol, k=1)
            self.s_regs[0] = spike
        else:
            self.u_regs[t] = self.leak * self.u_regs[t-1] * (1 - self.s_regs[t-1]) + inp
            vol = self.u_regs[t] - self.thresh
            spike = spike_function(vol, k=1)
            self.s_regs[t] = spike

        return spike

        

def spike_function(x, k):
    x_fwd = torch.sign(x) * 0.5 + 0.5
    x_bwd = torch.clamp(1 - k*x.abs(), min=0.)
    return (x_fwd - x_bwd).detach() + x_bwd

In [328]:
inp = torch.rand(1,28,28)
inp = inp.view(1,-1)
toy = model()
out = 0
for t in range(5):
    spike_inp = PoissonGen(inp)
    out = out + toy(spike_inp,t)
out

tensor([[3.6845, 1.4626]], grad_fn=<AddBackward0>)

In [318]:
du_regs_out[-1] = torch.exp(u_regs_out[-1])/torch.sum(u_regs_out[-1],1)
du_regs_2[-1] = torch.matmul(du_regs_out[-1],fc_out.weight)*de_func(u_regs_2[-1])
du_regs_1[-1] = torch.matmul(du_regs_2[-1],fc_2.weight)*de_func(u_regs_1[-1])

w_inp_1 = torch.matmul(torch.transpose(du_regs_1[-1],0,1),s_regs_inp[-1])
fc_1.weight.data += w_inp_1

w_1_2 = torch.matmul(torch.transpose(du_regs_2[-1],0,1),s_regs_1[-1])
fc_2.weight.data += w_1_2

w_2_out = torch.matmul(torch.transpose(du_regs_out[-1],0,1),s_regs_2[-1])
fc_out.weight.data += w_2_out


In [320]:
for t in range(time_step-2,-1,-1):
    print(t)

3
2
1
0


In [284]:
torch.tensor([[1],[0]]) + torch.tensor([[2],[2]])

tensor([[3],
        [2]])

In [313]:
type(nn.Parameter(w_inp_1))

torch.nn.parameter.Parameter